In [2]:
import os
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from ipywidgets import Text, Button, Output, HBox, VBox, Layout, HTML
from IPython.display import display, clear_output

MODEL_CACHE_PATH = "./f11_model"  # Replace with your actual model path

class ChatBotModel:
    def __init__(self, model_path: str = MODEL_CACHE_PATH) -> None:
        self.device = torch.device("cpu")  # You can modify this to use "xpu" if available
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=6)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model.to(self.device).eval()
        self.emotions = {
            0: ("sadness", "You are feeling sad."),
            1: ("joy", "You are feeling joyful! I'm glad to hear that."),
            2: ("love", "You are feeling love! Love is such a beautiful emotion. Cherish those feelings."),
            3: ("anger", "You are feeling angry. It's okay to feel angry sometimes."),
            4: ("fear", "You are feeling fear. Fear can be tough to handle."),
            5: ("surprise", "You are feeling surprised. Wow! That sounds surprising.")
        }

    def get_intro_message(self):
        return "Hai , I'm an emotional recognition chatbot. I can read your text and understand how you're feeling."

    def predict_emotion(self, user_input: str):
        inputs = self.tokenizer(user_input, padding='max_length', truncation=True, return_tensors='pt')
        inputs = {key: value.to(self.device) for key, value in inputs.items()}
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        predicted_class = logits.argmax(dim=-1).item()
        return self.emotions[predicted_class]

model_cache = {}

# Define user and bot icons
user_icon = "😀"
bot_icon = "🤖"

def interact_with_llm():
    def display_message(message, icon):
        formatted_message = f"<p><span style='color: {'#1e90ff' if icon == user_icon else '#228b22'};'>{icon} {message}</span></p>"
        chat_history.value += formatted_message
    
    chat_history = HTML(value="")
    user_input_widget = Text(placeholder="Type your message here...", layout=Layout(width='80%'))
    send_button = Button(description="Send", button_style="primary", layout=Layout(width='10%'))
    chat_spin = HTML(value="")
    spin_style = """
    <div class="loader"></div>
    <style>
    .loader {
      border: 5px solid #f3f3f3;
      border-radius: 50%;
      border-top: 5px solid #3498db;
      width: 8px;
      height: 8px;
      animation: spin 3s linear infinite;
    }
    @keyframes spin {
      0% { transform: rotate(0deg); }
      100% { transform: rotate(360deg); }
    }
    </style>
    """

    # Display bot introduction message initially
    bot_intro_message = ChatBotModel().get_intro_message()
    display_message(f"Bot: {bot_intro_message}", bot_icon)

    def display_user_input_widgets():
        display(VBox([chat_history, HBox([chat_spin, user_input_widget, send_button])]))

    display_user_input_widgets()

    def on_send(button):
        send_button.button_style = "warning"
        chat_spin.value = spin_style
        user_input = user_input_widget.value
        display_message(f"You: {user_input}", user_icon)
        
        if user_input.lower() == "exit":
            return

        model_key = "emotion_detection"
        if model_key not in model_cache:
            model_cache[model_key] = ChatBotModel(model_path=MODEL_CACHE_PATH)
        bot = model_cache[model_key]
        
        # Predict emotion and display bot response
        predicted_emotion = bot.predict_emotion(user_input)
        display_message(f"Bot: {predicted_emotion[1]}", bot_icon)
        
        send_button.button_style = "success"
        chat_spin.value = ""
        user_input_widget.value = ""

    send_button.on_click(on_send)

interact_with_llm()
